In [3]:
# Load all required libraries for the notebook, including data package
if(!require("opendatatoronto")) {
    install.packages("opendatatoronto")
    library(opendatatoronto) # data source
}
if(!require("tidyjson")) {
    install.packages("tidyjson")
    library(tidyjson)
}
if(!require("tidygeocoder")) {
    install.packages("tidygeocoder")
    library(tidygeocoder)
}
if(!require("sf")) {
    install.packages("sf")
    library(sf)
}

if(!require("ggspatial")) {
    install.packages("ggspatial")
    library(ggspatial)
}
if(!require("stringr")) {
    install.packages("stringr")
    library(stringr)
}
if(!require("dplyr")) {
    install.packages("dplyr")
    library(dplyr)
}
if(!require("ggplot2")) {
    install.packages("ggplot2")
    library(ggplot2)
}
if(!require("tidyverse")) {
    install.packages("tidyverse")
    library(tidyverse)
}
if(!require("FNN")) {
    install.packages("FNN")
    library(FNN)
}

if(!require("mapview")) {
    install.packages("mapview")
    library(mapview)
}

if(!require("forecast")) {
    install.packages("forecast")
    library(forecast)
}

if(!require("purrr")) {
    install.packages("purrr")
    library(purrr)
}

if(!require("e1071")) {
    install.packages("e1071")
    library(e1071)
}


Loading required package: opendatatoronto



Loading required package: tidyjson


Attaching package: 'tidyjson'


The following object is masked from 'package:stats':

    filter


Loading required package: tidygeocoder

Loading required package: sf

Linking to GEOS 3.11.2, GDAL 3.8.2, PROJ 9.3.1; sf_use_s2() is TRUE

Loading required package: ggspatial

Loading required package: stringr

Loading required package: dplyr


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Loading required package: ggplot2

Loading required package: tidyverse

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.4
✔ lubridate 1.9.2     ✔ tibble    3.2.1
✔ purrr     1.0.2     ✔ tidyr     1.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks tidyjson::filter(), stats::filter()
✖ dpl

In [4]:
# Get Data - Traffic
# output Data Description:
# Dataframe with all intersection and daily count ( peak 4 hours), including lng / lat

# # ? and todo:
# # is separate street name needed
# # direction of the street to be determined. How?


# # package_traffic <- show_package("traffic-volumes-at-intersections-for-all-modes")

# # get all resources for this package
# resources <- list_package_resources("traffic-volumes-at-intersections-for-all-modes")

# # identify datastore resources; by default, Toronto Open Data sets datastore resource format to CSV for non-geospatial and GeoJSON for geospatial resources
# datastore_resources <- filter(resources, tolower(format) %in% c("csv", "geojson"))

# # load data # The method of loading data directly using turns out to be insufficient as the get_resource() only returns first 32000 rows of record. 
# location <- filter(datastore_resources, row_number() == 1) %>% get_resource()
# traffic1 <- filter(datastore_resources, row_number() == 3) %>% get_resource()
# traffic2 <- filter(datastore_resources, row_number() == 4) %>% get_resource()
# traffic3 <- filter(datastore_resources, row_number() == 5) %>% get_resource()
# traffic4 <- filter(datastore_resources, row_number() == 6) %>% get_resource()
# traffic5 <- filter(datastore_resources, row_number() == 7) %>% get_resource()

# To Run the code download the raw files from 
# https://open.toronto.ca/dataset/traffic-volumes-at-intersections-for-all-modes/ 
# and save the files in .csv format to the path: ../Data/Toronto/Traffic, 7 Files below. 
# count_metadata.csv
# locations.csv
# raw-data-1980-1989.csv
# raw-data-1990-1999.csv
# raw-data-2000-2009.csv
# raw-data-2010-2019.csv
# raw-data-2020-2029.csv
location <- read.csv("../Data/Toronto/Traffic/locations.csv") # ensure these path are correct if you have to download the files manually. 
traffic1 <- read.csv("../Data/Toronto/Traffic/raw-data-1980-1989.csv")
traffic2 <- read.csv("../Data/Toronto/Traffic/raw-data-1990-1999.csv")
traffic3 <- read.csv("../Data/Toronto/Traffic/raw-data-2000-2009.csv")
traffic4 <- read.csv("../Data/Toronto/Traffic/raw-data-2010-2019.csv")
traffic5 <- read.csv("../Data/Toronto/Traffic/raw-data-2020-2029.csv")
all_traffic = bind_rows(traffic1,traffic2,traffic3,traffic4,traffic5) # combine all raw data into one data frame

In [249]:
# clean and transform load - Traffic Data
# Output data for modelling CleanTraffic
# define parameters for cleaning
peakhours <- 4 # number of peak hours of data per day. value should be between 1 and 10

# get full intersection volume for each intersection based on peak hours of each day. 
# get average vol per intersection per year. 
# get number of years list, sort from low to high

# 

CleanTraffic <- all_traffic %>%
  filter(centreline_type == 2) %>% # only need intersection data
  mutate(counthour = str_extract(time_start, "(?<=T)(\\d+)(?=\\:)")) %>% # extract hour
  mutate(total_int_traffic = sb_cars_r + sb_cars_t + sb_cars_l +
    nb_cars_r + nb_cars_t + nb_cars_l + wb_cars_r + wb_cars_t +
    wb_cars_l + eb_cars_r + eb_cars_t + eb_cars_l) %>% # get total sum
  select(one_of(c(
    "count_date", "location_id", "location", "lng", "lat", "counthour",
    "total_int_traffic"
  ))) %>% # remove raw attributes, retain aggregate only
  group_by(across(all_of(c("count_date", "location_id", "location", "lng", "lat", "counthour")))) %>%
  summarise_at("total_int_traffic",sum) %>% # agregate hourly volume
  group_by(across(all_of(c("count_date", "location_id", "location", "lng", "lat")))) %>%
  slice_max(order_by = total_int_traffic, n = peakhours) %>% # filter top peak hour volume
  group_by(across(all_of(c("count_date", "location_id", "location", "lng", "lat")))) %>%
  summarise_at("total_int_traffic", sum)%>% # aggregate daily peak hour volume
  mutate(count_date= as.Date(count_date, format("%Y-%m-%d"))) %>% 
  mutate(year = as.numeric(format(count_date,'%Y'))) %>% #add year
  group_by(across(all_of(c("year", "location_id", "location", "lat","lng")))) %>% # group by year to get the average per intersection per year
  summarise(AvgTotal = mean(total_int_traffic), .groups = "drop")  # average traffic volume for that year and location
traffic_years <- unique(CleanTraffic$year) # get number of years list, sort from low to high

In [250]:
# Get Green P Parking package from Open Data-Toronto
package <- show_package("b66466c3-69c8-4825-9c8b-04b270069193")

data=as.data.frame('Green P Parking')  # read the dataset Green P Parking in the package 
data<-show_package(package)

resources<-list_package_resources(package)

# Identify resources
data_resources <- filter(resources, tolower(format) %in% c("csv", "json"))

# Load Green P Parking 2019 data
data <- filter(data_resources, row_number() == 1) %>% get_resource()

# Extract required columns from main data
data<- data.frame(address=data$carparks$address,
                                     lat=data$carparks$lat,
                                     lng=data$carparks$lng,
                                      carpark_type=data$carparks$carpark_type_str,
                                      rate_half_hr=data$carparks$rate_half_hour,
                                      capacity=data$carparks$capacity,
                                      rate=data$carparks$rate_half_hour
                  )

# Check class of each attribute
sapply(data, class) 

# Convert char to numeric class
data$lat<-as.numeric(data$lat)
data$lng<-as.numeric(data$lng)
data$rate_half_hr<-as.numeric(data$rate_half_hr)
data$capacity<-as.numeric(data$capacity)

sapply(data, class) 

# Check for missing values
any(is.na(data))

# Extract street name from address
data <- data %>%
  mutate(extracted_address = str_replace_all(data$address, "\\(.*?\\)",""))

data$extracted_address<-str_replace_all(data$extracted_address, "-.*", "")
data$extracted_address<-str_replace_all(data$extracted_address, ",.*", "")

# Extract data with carpark_type as 'Surface'
data <- data %>%
  filter(carpark_type == "Surface")



address          lat          lng carpark_type rate_half_hr     capacity 
 "character"  "character"  "character"  "character"  "character"  "character" 
        rate 
 "character"

address          lat          lng carpark_type rate_half_hr     capacity 
 "character"    "numeric"    "numeric"  "character"    "numeric"    "numeric" 
        rate 
 "character"

[1] FALSE

In [251]:
# Extracting address, latitude, and longitude
data <- data[, c("address", "lat", "lng")]

# Adding the new rows
new_rows <- data.frame(
  address = c(
    "365 Lippincott Street",
    "35 Erindale Avenue",
    "14 Arundel Avenue",
    "265 Armadale Avenue",
    "1612 Danforth Avenue",
    "117 Hammersmith Avenue",
    "166 Woodbine Ave",
    "19 Spadina Road",
    "2300 Lake shore Boulevard West",
    "35 Erindale Avenue",
    "265 Armadale Avenue",
    "2300 Lake shore Boulevard West"
  ),
  lat = c(
    43.665054,
    43.688543,
    43.695882,
    43.721568,
    43.684903,
    43.693819,
    43.674124,
    43.677754,
    43.623545,
    43.688543,
    43.721568,
    43.623545
  ),
  lng = c(
    -79.409662,
    -79.411305,
    -79.42134,
    -79.427191,
    -79.325679,
    -79.428246,
    -79.319325,
    -79.403948,
    -79.479056,
    -79.411305,
    -79.427191,
    -79.479056
  )
)

# Adding the new rows to the extracted GreenPParking dataset
data <- rbind(data, new_rows)

# Creating the 'Convert' column
data$Convert <- ifelse(data$address %in% new_rows$address, 1, 0)

# Convert Lat/Lng to address
geo_rev_data<-data %>%
  tidygeocoder::reverse_geocode(
    lat=lat,
    long=lng,
    method="osm")



#write.csv(data, "GreenPParking.csv",row.names = FALSE)

In [ ]:
# Combine Parking Data and Clean Traffic Data to prepare input data for Time Series Model to predict traffic / EV charging demand

# loop through low to high year number
# filter each year, to list of unique intersections with traffic volume
# run knn algo against parking lot, find k nearst intersection
# use knn nn.index to find the k intersection vol, get average, creating a vector
# add vector to parking data as new column with year and volume
# next loop for the next year

#Output TS_Input

K <- 5 # Parameter used in knn calculation. 
All_park <- data %>%
    select(one_of(c("lat","lng"))) # location coordinates of all surface car parks. 

TS_Input <- data # TS_Input is the input data for the time series of traffic volume forecast. 

for (i in traffic_years) {
    # print(paste("This is year" ,i))
    year_traffic<- filter(CleanTraffic, year == i) # get the traffic volume, intersection for each year.
    All_int <-select(year_traffic,one_of(c("lat","lng"))) # extract just the coordinates for K nearest neighter calculation
    knn_dist <- get.knnx(All_int, All_park, k=K, algorithm="kd_tree") # get the K nearest intersection to the parking lot and their index list
    
    get.mean <- function(x) { # custom function defined to calculate the average of the K nearest intersection volume
    mean(slice(year_traffic, c(x))$AvgTotal)
    }

    TS_Input[paste0("NearbyTraffic_",i)]  <- apply(knn_dist$nn.index,1,get.mean) # calculate the avaerage per row of indices

}

In [ ]:
# Model 1 - Time Series Forecast

# forecast traffic for 2024 given the TS_Input
# apply % of EV vehicle multiplier for 2022 and predicted year 2024 to get amount of estimated EV traffic.
# data frame that has the original location, EV traffic for 2022, and predicted EV traffic for 2024


library(forecast)
library(purrr)

# Time series model and Forecast for 2024
arima_models <- apply(TS_Input[, -c(1,2,3,4,5,6,7,8)], 1, auto.arima)
model_forecasts_2024 <- lapply(arima_models, function(x) forecast(x, h = 1))

TS_Output_2024 <- map_dbl(model_forecasts_2024, "mean")
TS_Output_2024 <- round(TS_Output_2024)


# Format output for 2024
# Data frame that contains the original location, lat, and lng columns,
# % of EV traffic for 2022 and % of EV traffic for predicted 2024

head(TS_Output_2024)

# % of EV vehicle multiplier for 2022
mulitplier_2022 = .03

# % of EV Vehicle multiplier for predicted year 
multiplier_predicted = .0533

TS_Output_df = data.frame(TS_Input$address,TS_Input$lat,TS_Input$lng, TS_Input$NearbyTraffic_2022,TS_Output_2024)
#head(TS_Output_df)

# Apply % of EV Vehicle multiplier to 2022 and predicted year 2024, present a new columns to Df
traffic_2022 <- data.frame(TS_Output_df$TS_Input.NearbyTraffic_2022)
traffic_2024 <-data.frame(TS_Output_df$TS_Output_2024)

TS_Output_df$EV_nearbyTraffic_2022 <- apply(traffic_2022, 1, function(x) round(x * mulitplier_2022))
TS_Output_df$EV_nearbyTraffic_2024 <- apply(traffic_2024, 1, function(x) round(x * multiplier_predicted))


# Final Output
head(TS_Output_df)

In [ ]:
# Model 2 - SVM

# Pre-processing

# Read data
parking_data <- data
business_data <- read.csv("business.csv")

K <- 8  # Number of nearest businesses to consider

for (i in 1:nrow(parking_data)) {
    parking_spot <- parking_data[i, c("lat", "lng")]  # Coordinates of the parking spot
    nearest_biz_indices <- get.knnx(business_data[, c("lat", "long")], parking_spot, k = K, algorithm = "kd_tree")$nn.index
    nearest_biz_distances <- get.knnx(business_data[, c("lat", "long")], parking_spot, k = K, algorithm = "kd_tree")$nn.dist
    
    # Calculate the number of nearest businesses and their distances
    num_nearest_biz <- length(nearest_biz_indices)
    avg_distance <- mean(nearest_biz_distances)
    
    nearest_biz_customers <- sapply(nearest_biz_indices, function(idx) {
  mean(business_data[idx, "qCustomer"])
})
    nearest_biz_time_spent <- sapply(nearest_biz_indices, function(idx) {
  mean(business_data[idx, "tCustomer"])
})

    overall_mean_customers <- mean(nearest_biz_customers)
    overall_mean_time_spent <- mean(nearest_biz_time_spent)


    # Add the results to the parking_data dataframe
    parking_data[i, paste0("nearest_biz_count_", K)] <- num_nearest_biz
    parking_data[i, paste0("avg_distance_to_biz_", K)] <- avg_distance
    parking_data[i, paste0("avg_customers_nearby_", K)] <- mean(overall_mean_customers)
    parking_data[i, paste0("avg_time_spent_nearby_", K)] <- mean(overall_mean_time_spent)
}

# Add EV Traffic data
parking_data$traffic_volume=TS_Output_df$EV_nearbyTraffic_2022	

parking_data <- parking_data %>%
  group_by(address) %>%
  summarise (distance = mean(avg_distance_to_biz_8), n_business = mean(nearest_biz_count_8),n_customers = mean(avg_customers_nearby_8), time_spent= mean(avg_time_spent_nearby_8), traffic_volume = traffic_volume, convert= mean(Convert)) 

# Normalize predictors
parking_data$distance <- scale(parking_data$distance)
parking_data$traffic_volume <- scale(parking_data$traffic_volume)
parking_data$time_spent<- scale(parking_data$time_spent)
parking_data$n_customers<- scale(parking_data$n_customers)

set.seed(123)  # for reproducibility

train_indices <- sample(1:nrow(filtered_result_df3), 0.7 * nrow(parking_data))  # 70% for training
train_data <- parking_data[train_indices, ]
test_data <- parking_data[-train_indices, ]

# Create a formula including predictor columns
formula <- as.formula("convert ~ distance + traffic_volume ")

# Create the SVM model with the formula
svm_model <- svm(formula, data = train_data, kernel = "linear")

# Get the coefficients (weights) of the model
weights <- coef(svm_model)

# Coefficients from the SVM model
coefficients <- coef(svm_model)[-1]  # Exclude the intercept

# Select the columns for which to calculate the score
cols <- c("distance", "traffic_volume")

# Calculate the score for each row
parking_data$score <- rowSums(parking_data[cols] * coefficients)


# Sort the rows in descending order according to the score
sorted_result <- parking_data %>%
  arrange(desc(score))

# View the sorted result
head(sorted_result,12)


In [256]:
# Get Data - Intersection
# output Data Description:


In [257]:
# Get Data - Business
# output Data Description: 

In [258]:
# Define Region of Interest - Boundary
## coordinates manually looked up from location dataset
# 1406	5370	DUPONT ST AT OSSINGTON AVE (PX 842)	-79.429019	43.670031996501194
# 251	4180	DUPONT ST AT SPADINA RD (PX 840)	-79.407122	43.67485699954096
# 1885	5864	COLLEGE ST AT OSSINGTON AVE (PX 829)	-79.422705	43.65439999619167
# 241	4170	COLLEGE ST AT SPADINA AVE (PX 279)	-79.400048	43.65794800150128

# Input
# Output

boundary <- location %>%
  select(location_id, location, lng, lat) %>%
  filter(location_id %in% list(5370, 4180, 5864, 4170)) # boundary intersection ID

lng_min <- min(boundary$lng) # west most value since it's negative
lng_max <- max(boundary$lng) # east most value
lat_min <- min(boundary$lat) # south most value
lat_max <- max(boundary$lat) # north most value


In [ ]:
# Result and Discussion
